In [ ]:
import io
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

In [ ]:
import os  
import cv2
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

# Loading np array from images
def make_dataset(labels, dir_path, IMG_SIZE = 150):
    # initial an empty list X to store image of np.array()
    X = []

    # initial an empty list Z to store labels/names of cat individauls
    Z = []
    for label in labels:
        DIR = os.path.join(dir_path,label)
        for img in tqdm(os.listdir(DIR)):
            path = os.path.join(DIR,img)
            # reading images
            img = cv2.imread(path,cv2.IMREAD_COLOR)
            # resizing images to (150, 150, 3), 3 is the number of channels - RGB
            img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
            X.append(np.array(img))
            Z.append(str(label)) 
            
    ## Transform labels in Z to Y from class number
    le=LabelEncoder()
    Y=le.fit_transform(Z)

    ## Transform and normalize X in the range of [0, 1]
    X=np.array(X)
    X=X/255.
    return X, Y

In [ ]:
# dir_path = '/kaggle/input/auckland-zoom-monkeys/Auckland_Zoo_Monkeys/'
dir_path = "F:\\Adam\\Pictures\\AucklandZooImages\\labelled_individuals\\Extracted"

labels = ['Arani', 'Inti', 'Ocuri', 'Poco', 'Rattaplan', 'Romy']

# randomly select seen datset and unseen dataset
# make open set and unseen set 
def make_seen_unseen(labels, num_seen):
    arr = np.arange(len(labels))
    np.random.shuffle(arr)
    
    labels_seen = [labels[i] for i in arr[:num_seen]]
    labels_unseen = [labels[i] for i in arr[num_seen:]]
    
    X_seen, Y_seen = make_dataset(labels_seen, dir_path)
    X_unseen, Y_unseen = make_dataset(labels_unseen, dir_path)
    Y_unseen = ['unseen'] * len(Y_unseen)
    Y_unseen = np.array(Y_unseen)
    return X_seen, Y_seen, X_unseen, Y_unseen

# Configuration settings

In [ ]:
imsize = 150
EPOCHS = 10
batch_size = 32
embeddingDim = 128
num_individuals = len(labels)

# number of folds in k-fold cross-validation
NUM_SPLITS = 3

In [ ]:
X_seen, Y_seen, X_unseen, Y_unseen = make_seen_unseen(labels, num_individuals - 1)

100%|██████████| 214/214 [00:00<00:00, 981.68it/s] 


# Train/test set split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_seen, Y_seen, test_size = 0.2, random_state=2021)

X_val_unseen, X_test_unseen, Y_val_unseen, Y_test_unseen = train_test_split(X_unseen, Y_unseen, test_size = 0.5, random_state=2021)

# Stratified k fold Cross Validation

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=NUM_SPLITS, random_state=2021, shuffle=True)

# VGG16 Classification Model

In [ ]:
def evaluate_vgg16(lr):
    base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                      input_shape = (imsize,imsize,3),
                      weights = 'imagenet')

    # freeze all the layers of VGG, so they won't be trained.
    for layer in base_model_16.layers:
        layer.trainable = False

    model_vgg_16 = tf.keras.models.Sequential([
        base_model_16,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=1024, activation='relu'),
        tf.keras.layers.Dense(units=num_individuals, activation='softmax')
    ])

    model_vgg_16.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='sparse_categorical_crossentropy', metrics=['acc']) 

    model_vgg_16.fit(x=x_train, y=y_train,
                     epochs=EPOCHS,
                     batch_size=batch_size,
                     verbose=1)
    
    acc = model_vgg_16.evaluate(x_val, y_val)[1]
    
    return acc

In [ ]:
lr = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]

terms = {}
for i in lr:
    terms['{}'.format(i)] = []

for train_index, test_index in skf.split(X_train, Y_train):
    x_train,x_val,y_train,y_val = X_train[train_index], X_train[test_index], Y_train[train_index], Y_train[test_index]
    
    for i in lr:
        terms['{}'.format(i)].append(evaluate_vgg16(lr = i))

Epoch 1/10
28/28 [==============================] - 6s 80ms/step - loss: 1.8041 - acc: 0.2737
Epoch 2/10
28/28 [==============================] - 2s 59ms/step - loss: 1.3471 - acc: 0.4369
Epoch 3/10
28/28 [==============================] - 2s 58ms/step - loss: 1.2179 - acc: 0.5515
Epoch 4/10
28/28 [==============================] - 2s 58ms/step - loss: 1.1364 - acc: 0.6031
Epoch 5/10
28/28 [==============================] - 2s 74ms/step - loss: 1.0470 - acc: 0.7053
Epoch 6/10
28/28 [==============================] - 2s 60ms/step - loss: 0.9594 - acc: 0.7210
Epoch 7/10
28/28 [==============================] - 2s 58ms/step - loss: 0.8960 - acc: 0.7377
Epoch 8/10
28/28 [==============================] - 2s 58ms/step - loss: 0.8599 - acc: 0.7715
Epoch 9/10
28/28 [==============================] - 2s 58ms/step - loss: 0.8019 - acc: 0.7873
Epoch 10/10
14/14 [==============================] - 2s 170ms/step - loss: 0.8891 - acc: 0.7136
Epoch 1/10
28/28 [==============================] - 2s 57m

In [ ]:
import pandas as pd
acc = pd.DataFrame(terms)
table_vgg16 = acc.agg(['mean', 'std']).T
table_vgg16

,mean,std
1e-05,0.739591,0.032459
0.0001,0.892909,0.008801
0.001,0.882887,0.021421
0.01,0.899834,0.013488
0.1,0.325115,0.000901


# Retrain VGG16 model on training set + val set, test it on test set

In [ ]:
# choose best lr
lr = 0.0001 

base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                      input_shape = (imsize,imsize,3),
                      weights = 'imagenet')

# freeze all the layers of VGG, so they won't be trained.
for layer in base_model_16.layers:
    layer.trainable = False

model_vgg_16 = tf.keras.models.Sequential([
    base_model_16,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1024, activation='relu'),
    tf.keras.layers.Dense(units=num_individuals, activation='softmax')
])

model_vgg_16.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='sparse_categorical_crossentropy', metrics=['acc']) 

model_vgg_16.fit(x=X_train, y=Y_train,
                 epochs=EPOCHS,
                 batch_size=batch_size,
                 verbose=1)

Epoch 1/10
41/41 [==============================] - 5s 111ms/step - loss: 1.4395 - acc: 0.4032
Epoch 2/10
41/41 [==============================] - 3s 73ms/step - loss: 0.6429 - acc: 0.8179
Epoch 3/10
41/41 [==============================] - 3s 75ms/step - loss: 0.4120 - acc: 0.9050
Epoch 4/10
41/41 [==============================] - 3s 69ms/step - loss: 0.2526 - acc: 0.9626
Epoch 5/10
41/41 [==============================] - 3s 76ms/step - loss: 0.1821 - acc: 0.9837
Epoch 6/10
41/41 [==============================] - 3s 64ms/step - loss: 0.1360 - acc: 0.9930
Epoch 7/10
41/41 [==============================] - 3s 66ms/step - loss: 0.1051 - acc: 0.9993
Epoch 8/10
41/41 [==============================] - 3s 63ms/step - loss: 0.0879 - acc: 0.9962
Epoch 9/10
41/41 [==============================] - 3s 68ms/step - loss: 0.0703 - acc: 1.0000
Epoch 10/10
41/41 [==============================] - 3s 68ms/step - loss: 0.0552 - acc: 1.0000


In [ ]:
acc_vgg16 = round(model_vgg_16.evaluate(X_test, Y_test)[1], 2)

print('Accuracy of VGG16 on the test set is {}'.format(acc_vgg16))

11/11 [==============================] - 2s 137ms/step - loss: 0.2335 - acc: 0.9323
Accuracy of VGG16 on the test set is 0.93


# Contrastive Loss

In [ ]:
# Helper Function 

# Create positive pairs and negative pairs
import random
def create_pairs(images, labels):
    numClasses = len(labels)
    # initialize two empty lists to hold the (image, image) pairs and
    # labels to indicate if a pair is positive (0) or negative (1)
    np.random.seed(2021)
    pairImages = []
    pairLabels = []
    
    # calculate the total number of classes present in the dataset
    # and then build a list of indexes for each class label that
    # provides the indexes for all examples with a given label
    idx = [np.where(labels == i)[0] for i in range(numClasses)]
    
    # loop voer all images
    for idxA in range(len(images)):
        # grab the current image and label belonging to the current iteration
        currentImage = images[idxA]
        label = labels[idxA]
        
        # randomly pick on an image that belongs to the *same* class label
        posId = random.choice(idx[label])
        posImage = images[posId]
        
        # prepare a positive pair and update the images and labels
        pairImages.append([currentImage, posImage])
        pairLabels.append([0])
        
        # grab the indices for each of the class labels *not* equal to
        # the current label and randomly pick an image corresponding
        # to a label *not* equal to the current label
        negId = np.where(labels != label)[0]
        negIdx = random.choice(negId)
        negImage = images[negIdx]
        
        # prepare a negative pair of images and update out lists
        pairImages.append([currentImage, negImage])
        pairLabels.append([1])
    
    return (np.array(pairImages), np.array(pairLabels))



# Function to calculate the distance between two images (Euclidean Distance used here)
import tensorflow.keras.backend as K
def euclidean_distance(vectors):
    # unpack the vectors into separate lists
    (featsA, featsB) = vectors
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1,
                       keepdims=True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))


# contrastive loss function
def contrastive_loss(y, preds, margin=1):
    # explicitly cast the true class label data type to the predicted
    # class label data type (otherwise we run the risk of having two
    # separate data types, causing TensorFlow to error out)
    y = tf.cast(y, preds.dtype)
    # calculate the contrastive loss between the true labels and
    # the predicted labels
    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean((1 - y) * squaredPreds + y * squaredMargin)
    # return the computed contrastive loss to the calling function
    return loss

## Closed Set

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

def evaluate_cl_closed_set(lr, k):
    base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                      input_shape = (imsize,imsize,3),
                                                      weights = 'imagenet')
    
    # freeze all the layers of VGG, so they won't be trained.
    for layer in base_model_16.layers:
        layer.trainable = False

    model_cl = tf.keras.models.Sequential([
        base_model_16,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=1024, activation='relu'),
        tf.keras.layers.Dense(embeddingDim, activation=None), # No activation on final dense layer
        tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
    ])

    
    imgA = tf.keras.layers.Input(shape=((imsize, imsize, 3)))
    imgB = tf.keras.layers.Input(shape=((imsize, imsize, 3)))
    
    featsA = model_cl(imgA)
    featsB = model_cl(imgB)
   
    distance = tf.keras.layers.Lambda(euclidean_distance)([featsA, featsB])
    model = tf.keras.Model(inputs=[imgA, imgB], outputs=distance)
    model.compile(loss=contrastive_loss, optimizer=tf.keras.optimizers.Adam(lr))

    model.fit([pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:],
              batch_size = batch_size,
              epochs=EPOCHS, 
              verbose=1)
    
    embedding_train_cl = []
    for i in range(len(y_train)):
        embedding_train_cl.append(model_cl(x_train[i].reshape(1, imsize, imsize, 3))[0])
    embedding_train_cl = np.array(embedding_train_cl, dtype=float) 

    knn_cl = KNeighborsClassifier(n_neighbors = k)
    knn_cl.fit(embedding_train_cl, y_train)
    
    # Fixes OOM issues
    x_test_embedding = model_cl.predict(x_val)
    acc = round(knn_cl.score(x_test_embedding, y_val), 2)
    print('Accuracy on the val set with contrastive loss is {}'.format(acc))
    
    return acc

In [ ]:
lr = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
k = [1, 3, 5]

terms = {}
for i in lr:
    for j in k:
        terms['{}_{}'.format(i, j)] = []

for train_index, test_index in skf.split(X_train, Y_train):
    x_train,x_val,y_train,y_val = X_train[train_index], X_train[test_index], Y_train[train_index], Y_train[test_index]
    
    (pairTrain, labelTrain) = create_pairs(x_train, y_train)
    
    for i in lr:
        for j in k:
            terms['{}_{}'.format(i, j)].append(evaluate_cl_closed_set(lr = i, k = j))

Epoch 1/10
55/55 [==============================] - 9s 143ms/step - loss: 0.2482
Epoch 2/10
55/55 [==============================] - 8s 140ms/step - loss: 0.2357
Epoch 3/10
55/55 [==============================] - 8s 137ms/step - loss: 0.2255
Epoch 4/10
55/55 [==============================] - 7s 134ms/step - loss: 0.2123
Epoch 5/10
55/55 [==============================] - 7s 133ms/step - loss: 0.2035
Epoch 6/10
55/55 [==============================] - 8s 139ms/step - loss: 0.1966
Epoch 7/10
55/55 [==============================] - 8s 136ms/step - loss: 0.1841
Epoch 8/10
55/55 [==============================] - 12s 218ms/step - loss: 0.1776
Epoch 9/10
55/55 [==============================] - 9s 155ms/step - loss: 0.1654
Epoch 10/10
55/55 [==============================] - 7s 131ms/step - loss: 0.1617
Accuracy on the val set with contrastive loss is 0.85
Epoch 1/10
55/55 [==============================] - 11s 182ms/step - loss: 0.2487
Epoch 2/10
55/55 [==============================] - 

In [ ]:
import pandas as pd
acc = pd.DataFrame(terms)
table_cl_closed_set = acc.agg(['mean', 'std']).T
table_cl_closed_set

,mean,std
1e-05_1,0.850000,0.010000
1e-05_3,0.826667,0.011547
1e-05_5,0.816667,0.005774
0.0001_1,0.900000,0.010000
0.0001_3,0.906667,0.005774
0.0001_5,0.893333,0.015275
0.001_1,0.873333,0.011547
0.001_3,0.883333,0.020817
0.001_5,0.880000,0.010000
0.01_1,0.796667,0.020817


In [ ]:
max_row = table_cl_closed_set['mean'].argmax()
table_cl_closed_set.iloc[max_row]

mean    0.906667
std     0.005774
Name: 0.0001_3, dtype: float64

## Retrain Contrastive Loss on trainset + val set, and test it on test set

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# choose best lr and k
lr = 0.0001
k = 3

base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                  input_shape = (imsize,imsize,3),
                                                  weights = 'imagenet')
    
# freeze all the layers of VGG, so they won't be trained.
for layer in base_model_16.layers:
    layer.trainable = False

model_cl = tf.keras.models.Sequential([
    base_model_16,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1024, activation='relu'),
    tf.keras.layers.Dense(embeddingDim, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
])

    
imgA = tf.keras.layers.Input(shape=((imsize, imsize, 3)))
imgB = tf.keras.layers.Input(shape=((imsize, imsize, 3)))
    
featsA = model_cl(imgA)
featsB = model_cl(imgB)
   
distance = tf.keras.layers.Lambda(euclidean_distance)([featsA, featsB])
model = tf.keras.Model(inputs=[imgA, imgB], outputs=distance)
model.compile(loss=contrastive_loss, optimizer=tf.keras.optimizers.Adam(lr))

(pairTrain, labelTrain) = create_pairs(X_train, Y_train)

model.fit([pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:],
          batch_size = batch_size,
          epochs=EPOCHS, 
          verbose=1)
    
embedding_train_cl = []
for i in range(len(Y_train)):
        embedding_train_cl.append(model_cl(X_train[i].reshape(1, imsize, imsize, 3))[0])
embedding_train_cl = np.array(embedding_train_cl, dtype=float) 

knn_cl = KNeighborsClassifier(n_neighbors = k)
knn_cl.fit(embedding_train_cl, Y_train)
    
x_test_embedding = model_cl.predict(X_test)
acc_cl_closed_set = round(knn_cl.score(x_test_embedding, Y_test), 2)

print('Accuracy of Constractive Loss on test set is {}'.format(round(acc_cl_closed_set, 2)))

Epoch 1/10
82/82 [==============================] - 16s 188ms/step - loss: 0.2464
Epoch 2/10
82/82 [==============================] - 15s 182ms/step - loss: 0.1724
Epoch 3/10
82/82 [==============================] - 13s 160ms/step - loss: 0.1197
Epoch 4/10
82/82 [==============================] - 12s 153ms/step - loss: 0.0927
Epoch 5/10
82/82 [==============================] - 12s 144ms/step - loss: 0.0691
Epoch 6/10
82/82 [==============================] - 12s 143ms/step - loss: 0.0595
Epoch 7/10
82/82 [==============================] - 12s 144ms/step - loss: 0.0471
Epoch 8/10
82/82 [==============================] - 12s 141ms/step - loss: 0.0388
Epoch 9/10
82/82 [==============================] - 12s 145ms/step - loss: 0.0321
Epoch 10/10
82/82 [==============================] - 12s 146ms/step - loss: 0.0282
Accuracy of Constractive Loss on test set is 0.92


## Open Set

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

def evaluate_cl_open_set(lr=0.0001, k = 1, d_t = 0.5):
    base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                      input_shape = (imsize,imsize,3),
                                                      weights = 'imagenet')
    
    # freeze all the layers of VGG, so they won't be trained.
    for layer in base_model_16.layers:
        layer.trainable = False

    model_cl = tf.keras.models.Sequential([
        base_model_16,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=1024, activation='relu'),
        tf.keras.layers.Dense(embeddingDim, activation=None), # No activation on final dense layer
        tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
    ])

    
    imgA = tf.keras.layers.Input(shape=((imsize, imsize, 3)))
    imgB = tf.keras.layers.Input(shape=((imsize, imsize, 3)))
    
    featsA = model_cl(imgA)
    featsB = model_cl(imgB)
   
    distance = tf.keras.layers.Lambda(euclidean_distance)([featsA, featsB])
    model = tf.keras.Model(inputs=[imgA, imgB], outputs=distance)
    model.compile(loss=contrastive_loss, optimizer=tf.keras.optimizers.Adam(lr))

    model.fit([pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:],
              batch_size = batch_size,
              epochs=EPOCHS, 
              verbose=1)
    
    embedding_train_cl = []
    for i in range(len(y_train)):
        embedding_train_cl.append(model_cl(x_train[i].reshape(1, imsize, imsize, 3))[0])
    embedding_train_cl = np.array(embedding_train_cl, dtype=float) 

    knn_cl = KNeighborsClassifier(n_neighbors = k)
    knn_cl.fit(embedding_train_cl, y_train)
    
    #find the center point for each class in training set
    support_cl = []
    for i in np.unique(y_train):
        support_cl.append(np.mean(embedding_train_cl[y_train==i], axis=0))
        
    support_cl = np.array(support_cl, dtype=float)
    
    pred = []
    temp_x = np.append(x_val, X_val_unseen, axis=0)
    temp_y = np.append(y_val, Y_val_unseen, axis=0)
    
    arr = np.arange(temp_y.shape[0])
    np.random.shuffle(arr)
    
    temp_x = temp_x[arr]
    temp_y = temp_y[arr]
    
    for i in range(len(temp_y)):
        dists = []
        for j in range(len(np.unique(y_train))):
            embedding_test = model_cl(temp_x[i].reshape(1, 150, 150, 3))
            embedding_anchor = support_cl[j]
            dist = np.sum((embedding_test - embedding_anchor) ** 2) ** (1/2)
            dists.append(dist)
        if min(dists) >= d_t:
            pred.append('unseen')
        else:
            pred.append(knn_cl.predict(embedding_test)[0])

    pred = np.array(pred)
    
    acc_open = round(np.mean(pred == temp_y), 2)
    print('The accuracy on the val set with Contrastive Loss is {}'.format(acc_open))
    
    return acc_open

In [ ]:
d_t = [0.4, 0.5, 0.6, 0.7, 0.8]

terms = {}
for i in d_t:
    terms['{}'.format(i)] = []

for train_index, test_index in skf.split(X_train, Y_train):
    x_train,x_val,y_train,y_val = X_train[train_index], X_train[test_index], Y_train[train_index], Y_train[test_index]
    
    (pairTrain, labelTrain) = create_pairs(x_train, y_train)
    
    for i in d_t:
        terms['{}'.format(i)].append(evaluate_cl_open_set(d_t = i))

Epoch 1/10
55/55 [==============================] - 10s 163ms/step - loss: 0.2517
Epoch 2/10
55/55 [==============================] - 9s 170ms/step - loss: 0.1974
Epoch 3/10
55/55 [==============================] - 8s 152ms/step - loss: 0.1443
Epoch 4/10
55/55 [==============================] - 8s 154ms/step - loss: 0.1145
Epoch 5/10
55/55 [==============================] - 10s 190ms/step - loss: 0.0927
Epoch 6/10
55/55 [==============================] - 7s 121ms/step - loss: 0.0708
Epoch 7/10
55/55 [==============================] - 7s 120ms/step - loss: 0.0628
Epoch 8/10
55/55 [==============================] - 7s 120ms/step - loss: 0.0533
Epoch 9/10
55/55 [==============================] - 7s 120ms/step - loss: 0.0457
Epoch 10/10
55/55 [==============================] - 7s 120ms/step - loss: 0.0363
The accuracy on the val set with Contrastive Loss is 0.74
Epoch 1/10
55/55 [==============================] - 7s 113ms/step - loss: 0.2558
Epoch 2/10
55/55 [==============================

ipykernel_launcher:75: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


The accuracy on the val set with Contrastive Loss is 0.0
Epoch 1/10
55/55 [==============================] - 7s 111ms/step - loss: 0.2516
Epoch 2/10
55/55 [==============================] - 6s 112ms/step - loss: 0.1887
Epoch 3/10
55/55 [==============================] - 11s 194ms/step - loss: 0.1408
Epoch 4/10
55/55 [==============================] - 10s 188ms/step - loss: 0.1125
Epoch 5/10
55/55 [==============================] - 6s 115ms/step - loss: 0.0932
Epoch 6/10
55/55 [==============================] - 6s 114ms/step - loss: 0.0817
Epoch 7/10
55/55 [==============================] - 6s 114ms/step - loss: 0.0621
Epoch 8/10
55/55 [==============================] - 6s 112ms/step - loss: 0.0525
Epoch 9/10
55/55 [==============================] - 6s 110ms/step - loss: 0.0458
Epoch 10/10
55/55 [==============================] - 6s 110ms/step - loss: 0.0377


ipykernel_launcher:75: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


The accuracy on the val set with Contrastive Loss is 0.0
Epoch 1/10
55/55 [==============================] - 7s 116ms/step - loss: 0.2545
Epoch 2/10
55/55 [==============================] - 6s 111ms/step - loss: 0.1879
Epoch 3/10
55/55 [==============================] - 6s 110ms/step - loss: 0.1417
Epoch 4/10
55/55 [==============================] - 6s 115ms/step - loss: 0.1100
Epoch 5/10
55/55 [==============================] - 7s 119ms/step - loss: 0.0887
Epoch 6/10
55/55 [==============================] - 9s 162ms/step - loss: 0.0742
Epoch 7/10
55/55 [==============================] - 8s 148ms/step - loss: 0.0653
Epoch 8/10
55/55 [==============================] - 7s 119ms/step - loss: 0.0527
Epoch 9/10
55/55 [==============================] - 6s 114ms/step - loss: 0.0488
Epoch 10/10
55/55 [==============================] - 6s 111ms/step - loss: 0.0396
The accuracy on the val set with Contrastive Loss is 0.7
Epoch 1/10
55/55 [==============================] - 8s 132ms/step - loss: 0

...

ipykernel_launcher:75: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


The accuracy on the val set with Contrastive Loss is 0.0
Epoch 1/10
55/55 [==============================] - 9s 139ms/step - loss: 0.2529
Epoch 2/10
55/55 [==============================] - 8s 142ms/step - loss: 0.2041
Epoch 3/10
55/55 [==============================] - 8s 140ms/step - loss: 0.1499
Epoch 4/10
55/55 [==============================] - 8s 140ms/step - loss: 0.1138
Epoch 5/10
55/55 [==============================] - 8s 143ms/step - loss: 0.0938
Epoch 6/10
55/55 [==============================] - 8s 144ms/step - loss: 0.0752
Epoch 7/10
55/55 [==============================] - 8s 144ms/step - loss: 0.0635
Epoch 8/10
55/55 [==============================] - 8s 142ms/step - loss: 0.0484
Epoch 9/10
55/55 [==============================] - 8s 139ms/step - loss: 0.0407
Epoch 10/10
55/55 [==============================] - 8s 140ms/step - loss: 0.0363
The accuracy on the val set with Contrastive Loss is 0.75
Epoch 1/10
55/55 [==============================] - 8s 140ms/step - loss: 

ipykernel_launcher:75: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


The accuracy on the val set with Contrastive Loss is 0.0
Epoch 1/10
55/55 [==============================] - 9s 142ms/step - loss: 0.2554
Epoch 2/10
55/55 [==============================] - 8s 143ms/step - loss: 0.1938
Epoch 3/10
55/55 [==============================] - 8s 141ms/step - loss: 0.1471
Epoch 4/10
55/55 [==============================] - 8s 140ms/step - loss: 0.1139
Epoch 5/10
55/55 [==============================] - 8s 139ms/step - loss: 0.0890
Epoch 6/10
55/55 [==============================] - 8s 138ms/step - loss: 0.0746
Epoch 7/10
55/55 [==============================] - 8s 139ms/step - loss: 0.0594
Epoch 8/10
55/55 [==============================] - 8s 140ms/step - loss: 0.0510
Epoch 9/10
55/55 [==============================] - 8s 142ms/step - loss: 0.0431
Epoch 10/10
55/55 [==============================] - 8s 140ms/step - loss: 0.0343
The accuracy on the val set with Contrastive Loss is 0.0


ipykernel_launcher:75: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [ ]:
import pandas as pd
acc = pd.DataFrame(terms)
table_cl_open_set = acc.agg(['mean', 'std']).T
table_cl_open_set

,mean,std
0.4,0.730000,0.026458
0.5,0.723333,0.011547
0.6,0.710000,0.010000
0.7,0.000000,0.000000
0.8,0.000000,0.000000


## Retrain Contrastive Loss on trainset + val set, and test it on test set

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# choose the best value of d_t
lr = 0.0001
k = 3
d_t = 0.4

base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                  input_shape = (imsize,imsize,3),
                                                  weights = 'imagenet')
    
# freeze all the layers of VGG, so they won't be trained.
for layer in base_model_16.layers:
    layer.trainable = False

model_cl = tf.keras.models.Sequential([
    base_model_16,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1024, activation='relu'),
    tf.keras.layers.Dense(embeddingDim, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
])

imgA = tf.keras.layers.Input(shape=((imsize, imsize, 3)))
imgB = tf.keras.layers.Input(shape=((imsize, imsize, 3)))

featsA = model_cl(imgA)
featsB = model_cl(imgB)

distance = tf.keras.layers.Lambda(euclidean_distance)([featsA, featsB])

model = tf.keras.Model(inputs=[imgA, imgB], outputs=distance)

model.compile(loss=contrastive_loss, optimizer=tf.keras.optimizers.Adam(lr))

(pairTrain, labelTrain) = create_pairs(X_train, Y_train)

model.fit([pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:],
        batch_size = batch_size,
        epochs=EPOCHS, 
        verbose=1)

embedding_train_cl = []
for i in range(len(Y_train)):
    embedding_train_cl.append(model_cl(X_train[i].reshape(1, imsize, imsize, 3))[0])

embedding_train_cl = np.array(embedding_train_cl, dtype=float) 

knn_cl = KNeighborsClassifier(n_neighbors = k)
knn_cl.fit(embedding_train_cl, Y_train)
    
#find the center point for each class in training set
support_cl = []
for i in np.unique(Y_train):
    support_cl.append(np.mean(embedding_train_cl[Y_train==i], axis=0))
        
support_cl = np.array(support_cl, dtype=float)
    
pred = []
temp_x = np.append(X_test, X_val_unseen, axis=0)
temp_y = np.append(Y_test, Y_val_unseen, axis=0)
    
arr = np.arange(temp_y.shape[0])
np.random.shuffle(arr)
    
temp_x = temp_x[arr]
temp_y = temp_y[arr]
    
for i in range(len(temp_y)):
    dists = []
    for j in range(len(np.unique(Y_train))):
        embedding_test = model_cl(temp_x[i].reshape(1, 150, 150, 3))
        embedding_anchor = support_cl[j]
        dist = np.sum((embedding_test - embedding_anchor) ** 2) ** (1/2)
        dists.append(dist)
    if min(dists) >= d_t:
            pred.append('unseen')
    else:
        pred.append(knn_cl.predict(embedding_test)[0])

pred = np.array(pred)
    
acc_cl_open_set = round(np.mean(pred == temp_y), 2)
print('The accuracy on the test set with Open Dataset of Contrastive Loss is {}'.format(acc_cl_open_set))

Epoch 1/10
82/82 [==============================] - 17s 202ms/step - loss: 0.2521
Epoch 2/10
82/82 [==============================] - 13s 161ms/step - loss: 0.1779
Epoch 3/10
82/82 [==============================] - 13s 162ms/step - loss: 0.1241
Epoch 4/10
82/82 [==============================] - 15s 178ms/step - loss: 0.0969
Epoch 5/10
82/82 [==============================] - 19s 235ms/step - loss: 0.0780
Epoch 6/10
82/82 [==============================] - 12s 141ms/step - loss: 0.0620
Epoch 7/10
82/82 [==============================] - 14s 170ms/step - loss: 0.0522
Epoch 8/10
82/82 [==============================] - 10s 119ms/step - loss: 0.0404
Epoch 9/10
82/82 [==============================] - 12s 143ms/step - loss: 0.0364
Epoch 10/10
82/82 [==============================] - 16s 198ms/step - loss: 0.0322
The accuracy on the test set with Open Dataset of Contrastive Loss is 0.77


# Triplet Loss

## Closed Set

In [ ]:
def evaluate_tl_closed_set(lr, k):
    base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                      input_shape = (imsize, imsize, 3),
                                                      weights = 'imagenet')

    # freeze all the layers of VGG, so they won't be trained.
    for layer in base_model_16.layers:
        layer.trainable = False

    model_tl = tf.keras.models.Sequential([
        base_model_16,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=1024, activation='relu'),
        tf.keras.layers.Dense(embeddingDim, activation=None), # No activation on final dense layer
        tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
    ])

    model_tl.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss=tfa.losses.TripletSemiHardLoss())

    model_tl.fit(x=x_train, y= y_train,
                 batch_size=batch_size,
                 epochs=EPOCHS,
                 verbose=1) 
        
    embedding_train_tl = []
    for i in range(len(y_train)):
        embedding_train_tl.append(model_tl(x_train[i].reshape(1, imsize, imsize, 3))[0])
        
    embedding_train_tl = np.array(embedding_train_tl, dtype=float) 

    knn_tl = KNeighborsClassifier(n_neighbors = k)
    knn_tl.fit(embedding_train_tl, y_train)
    
    x_test_embedding = model_tl.predict(x_val)
    acc = round(knn_tl.score(x_test_embedding, y_val), 2)
    print('Accuracy on the the val set with Tripolet Loss is {}'.format(acc))
    
    return acc

In [ ]:
lr = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
k = [1, 3, 5]

batch_size = 64

terms = {}
for i in lr:
    for j in k:
        terms['{}_{}'.format(i, j)] = []
        
for train_index, test_index in skf.split(X_train, Y_train):
    x_train,x_val,y_train,y_val = X_train[train_index], X_train[test_index], Y_train[train_index], Y_train[test_index]
    
    for i in lr:
        for j in k:
            terms['{}_{}'.format(i, j)].append(evaluate_tl_closed_set(lr = i, k = j))

Epoch 1/10
14/14 [==============================] - 12s 383ms/step - loss: 0.9931
Epoch 2/10
14/14 [==============================] - 2s 161ms/step - loss: 0.9914
Epoch 3/10
14/14 [==============================] - 2s 156ms/step - loss: 0.9884
Epoch 4/10
14/14 [==============================] - 2s 160ms/step - loss: 0.9864
Epoch 5/10
14/14 [==============================] - 2s 142ms/step - loss: 0.9808
Epoch 6/10
14/14 [==============================] - 2s 156ms/step - loss: 0.9739
Epoch 7/10
14/14 [==============================] - 2s 154ms/step - loss: 0.9613
Epoch 8/10
14/14 [==============================] - 2s 158ms/step - loss: 0.9359
Epoch 9/10
14/14 [==============================] - 2s 163ms/step - loss: 0.9299
Epoch 10/10
14/14 [==============================] - 2s 156ms/step - loss: 0.9057
Accuracy on the the val set with Tripolet Loss is 0.86
Epoch 1/10
14/14 [==============================] - 3s 157ms/step - loss: 0.9930
Epoch 2/10
14/14 [==============================] - 

In [ ]:
import pandas as pd
acc = pd.DataFrame(terms)
table_tl_closed_set = acc.agg(['mean', 'std']).T
table_tl_closed_set

,mean,std
1e-05_1,0.866667,0.005774
1e-05_3,0.823333,0.023094
1e-05_5,0.820000,0.020000
0.0001_1,0.896667,0.015275
0.0001_3,0.890000,0.026458
0.0001_5,0.896667,0.011547
0.001_1,0.913333,0.011547
0.001_3,0.906667,0.005774
0.001_5,0.906667,0.005774
0.01_1,0.820000,0.034641


In [ ]:
max_row = table_cl_closed_set['mean'].argmax()
table_cl_closed_set.iloc[max_row]

mean    0.906667
std     0.005774
Name: 0.0001_3, dtype: float64

## Retrain Triplet Loss on trainset + val set, and test it on test set

In [ ]:
# choose best lr and k
lr = 0.001
k = 3

base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                  input_shape = (imsize, imsize, 3),
                                                  weights = 'imagenet')

# freeze all the layers of VGG, so they won't be trained.
for layer in base_model_16.layers:
        layer.trainable = False

model_tl = tf.keras.models.Sequential([
    base_model_16,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1024, activation='relu'),
    tf.keras.layers.Dense(embeddingDim, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
])

model_tl.compile(
    optimizer=tf.keras.optimizers.Adam(lr),
    loss=tfa.losses.TripletSemiHardLoss())

model_tl.fit(x=X_train, y= Y_train,
             batch_size=batch_size,
             epochs=EPOCHS,
             verbose=1) 
        
embedding_train_tl = []
for i in range(len(Y_train)):
    embedding_train_tl.append(model_tl(X_train[i].reshape(1, imsize, imsize, 3))[0])
        
embedding_train_tl = np.array(embedding_train_tl, dtype=float) 

knn_tl = KNeighborsClassifier(n_neighbors = k)
knn_tl.fit(embedding_train_tl, Y_train)
    
x_test_embedding = model_tl.predict(X_test)
acc_tl_closed_set = round(knn_tl.score(x_test_embedding, Y_test), 2)

print('Accuracy of Triplet Loss on the test set is {}'.format(round(acc_tl_closed_set, 2)))

Epoch 1/10
21/21 [==============================] - 3s 126ms/step - loss: 0.9786
Epoch 2/10
21/21 [==============================] - 3s 132ms/step - loss: 0.8951
Epoch 3/10
21/21 [==============================] - 3s 128ms/step - loss: 0.8096
Epoch 4/10
21/21 [==============================] - 3s 123ms/step - loss: 0.6041
Epoch 5/10
21/21 [==============================] - 3s 120ms/step - loss: 0.5338
Epoch 6/10
21/21 [==============================] - 3s 124ms/step - loss: 0.5247
Epoch 7/10
21/21 [==============================] - 3s 122ms/step - loss: 0.4735
Epoch 8/10
21/21 [==============================] - 3s 125ms/step - loss: 0.3427
Epoch 9/10
21/21 [==============================] - 3s 127ms/step - loss: 0.2737
Epoch 10/10
21/21 [==============================] - 3s 124ms/step - loss: 0.1773
Accuracy of Triplet Loss on the test set is 0.94


## Open Set

In [ ]:
def evaluate_tl_open_set(lr = 0.001, k = 1, d_t =0.5):
    base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                      input_shape = (imsize, imsize, 3),
                                                      weights = 'imagenet')

    # freeze all the layers of VGG, so they won't be trained.
    for layer in base_model_16.layers:
        layer.trainable = False

    model_tl = tf.keras.models.Sequential([
        base_model_16,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=1024, activation='relu'),
        tf.keras.layers.Dense(embeddingDim, activation=None), # No activation on final dense layer
        tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
    ])

    model_tl.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss=tfa.losses.TripletSemiHardLoss())

    model_tl.fit(x=x_train, y= y_train,
                 batch_size=batch_size,
                 epochs=EPOCHS,
                 verbose=1) 
        
    embedding_train_tl = []
    for i in range(len(y_train)):
        embedding_train_tl.append(model_tl(x_train[i].reshape(1, imsize, imsize, 3))[0])
        
    embedding_train_tl = np.array(embedding_train_tl, dtype=float) 

    knn_tl = KNeighborsClassifier(n_neighbors = k)
    knn_tl.fit(embedding_train_tl, y_train)
    
    #find the center point for each class in training set
    support_tl = []
    for i in np.unique(y_train):
        support_tl.append(np.mean(embedding_train_tl[y_train==i], axis=0))
        
    support_tl = np.array(support_tl, dtype=float)
    
    pred = []
    temp_x = np.append(x_val, X_unseen, axis=0)
    temp_y = np.append(y_val, Y_unseen, axis=0)
    
    arr = np.arange(temp_y.shape[0])
    np.random.shuffle(arr)
    
    temp_x = temp_x[arr]
    temp_y = temp_y[arr]
    
    for i in range(len(temp_y)):
        dists = []
        for j in range(len(np.unique(y_train))):
            embedding_test = model_tl(temp_x[i].reshape(1, 150, 150, 3))
            embedding_anchor = support_tl[j]
            dist = np.sum((embedding_test - embedding_anchor) ** 2) ** (1/2)
            dists.append(dist)
        if min(dists) >= d_t:
            pred.append('unseen')
        else:
            pred.append(knn_tl.predict(embedding_test)[0])

    pred = np.array(pred)
    
    acc_open = round(np.mean(pred == temp_y), 2)
    print('The accuracy on the Open Dataset with triplet loss is {}'.format(acc_open))
    
    return acc_open

In [ ]:
d_t = [0.4, 0.5, 0.6, 0.7, 0.8]

terms = {}
for i in d_t:
    terms['{}'.format(i)] = []

for train_index, test_index in skf.split(X_train, Y_train):
    x_train,x_val,y_train,y_val = X_train[train_index], X_train[test_index], Y_train[train_index], Y_train[test_index]
    
    for i in d_t:
        terms['{}'.format(i)].append(evaluate_tl_open_set(d_t = i))

Epoch 1/10
14/14 [==============================] - 2s 131ms/step - loss: 0.9837
Epoch 2/10
14/14 [==============================] - 2s 138ms/step - loss: 0.9554
Epoch 3/10
14/14 [==============================] - 2s 129ms/step - loss: 0.9171
Epoch 4/10
14/14 [==============================] - 2s 129ms/step - loss: 0.6885
Epoch 5/10
14/14 [==============================] - 2s 126ms/step - loss: 0.5913
Epoch 6/10
14/14 [==============================] - 2s 127ms/step - loss: 0.6157
Epoch 7/10
14/14 [==============================] - 2s 126ms/step - loss: 0.4602
Epoch 8/10
14/14 [==============================] - 2s 126ms/step - loss: 0.3853
Epoch 9/10
14/14 [==============================] - 2s 126ms/step - loss: 0.2776
Epoch 10/10
14/14 [==============================] - 2s 125ms/step - loss: 0.2769
The accuracy on the Open Dataset with triplet loss is 0.67
Epoch 1/10
14/14 [==============================] - 3s 127ms/step - loss: 0.9770
Epoch 2/10
14/14 [==============================]

In [ ]:
import pandas as pd
acc = pd.DataFrame(terms)
table_tl_open_set = acc.agg(['mean', 'std']).T
table_tl_open_set

,mean,std
0.4,0.660000,0.010000
0.5,0.680000,0.026458
0.6,0.720000,0.034641
0.7,0.703333,0.047258
0.8,0.596667,0.030551


In [ ]:
max_row = table_tl_open_set['mean'].argmax()
table_tl_open_set.iloc[max_row]

mean    0.720000
std     0.034641
Name: 0.6, dtype: float64

## Retrain Triplet Loss on trainset + val set, and test it on test set

In [53]:
lr = 0.001
k = 3
d_t = 0.6

base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                  input_shape = (imsize, imsize, 3),
                                                  weights = 'imagenet')

# freeze all the layers of VGG, so they won't be trained.
for layer in base_model_16.layers:
    layer.trainable = False

model_tl = tf.keras.models.Sequential([
    base_model_16,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1024, activation='relu'),
    tf.keras.layers.Dense(embeddingDim, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
])

model_tl.compile(
    optimizer=tf.keras.optimizers.Adam(lr),
    loss=tfa.losses.TripletSemiHardLoss())

model_tl.fit(x = X_train, y = Y_train,
             batch_size=batch_size,
             epochs=EPOCHS,
             verbose=1) 
        
embedding_train_tl = []
for i in range(len(Y_train)):
    embedding_train_tl.append(model_tl(X_train[i].reshape(1, imsize, imsize, 3))[0])
        
embedding_train_tl = np.array(embedding_train_tl, dtype=float) 

knn_tl = KNeighborsClassifier(n_neighbors = k)
knn_tl.fit(embedding_train_tl, Y_train)
    
#find the center point for each class in training set
support_tl = []
for i in np.unique(Y_train):
    support_tl.append(np.mean(embedding_train_tl[Y_train==i], axis=0))
        
support_tl = np.array(support_tl, dtype=float)
    
pred = []
temp_x = np.append(X_test, X_unseen, axis=0)
temp_y = np.append(Y_test, Y_unseen, axis=0)
    
arr = np.arange(temp_y.shape[0])
np.random.shuffle(arr)
    
temp_x = temp_x[arr]
temp_y = temp_y[arr]
    
for i in range(len(temp_y)):
    dists = []
    for j in range(len(np.unique(Y_train))):
        embedding_test = model_tl(temp_x[i].reshape(1, 150, 150, 3))
        embedding_anchor = support_tl[j]
        dist = np.sum((embedding_test - embedding_anchor) ** 2) ** (1/2)
        dists.append(dist)
    if min(dists) >= d_t:
        pred.append('unseen')
    else:
        pred.append(knn_tl.predict(embedding_test)[0])

pred = np.array(pred)
    
acc_tl_open_set = round(np.mean(pred == temp_y), 2)
print('The accuracy on the test set with Open Dataset of Triplet Loss is {}'.format(acc_tl_open_set))

Epoch 1/10
21/21 [==============================] - 3s 131ms/step - loss: 0.9765
Epoch 2/10
21/21 [==============================] - 3s 158ms/step - loss: 0.8963
Epoch 3/10
21/21 [==============================] - 3s 150ms/step - loss: 0.7312
Epoch 4/10
21/21 [==============================] - 3s 137ms/step - loss: 0.7007
Epoch 5/10
21/21 [==============================] - 3s 134ms/step - loss: 0.5574
Epoch 6/10
21/21 [==============================] - 3s 134ms/step - loss: 0.5054
Epoch 7/10
21/21 [==============================] - 4s 192ms/step - loss: 0.3745
Epoch 8/10
21/21 [==============================] - 3s 121ms/step - loss: 0.2922
Epoch 9/10
21/21 [==============================] - 2s 116ms/step - loss: 0.2336
Epoch 10/10
21/21 [==============================] - 2s 115ms/step - loss: 0.1395
The accuracy on the test set with Open Dataset of Triplet Loss is 0.76


# Summary on Closed Set

In [54]:
# After tuning hps (lr and k), the best accuracy on val set.
best_vgg16 = table_vgg16.loc['0.0001']
best_cl = table_cl_closed_set.loc['0.0001_3']
best_tl = table_tl_closed_set.loc['0.001_3']

pd.DataFrame({'VGG16_(LR = 0.0001)': best_vgg16,
              'Contrastive_Loss(LR = 0.0001, k = 3)': best_cl, 
              'Triplet_Loss(LR = 0.001, k = 3)': best_tl
             })

,VGG16_(LR = 0.0001),"Contrastive_Loss(LR = 0.0001, k = 3)","Triplet_Loss(LR = 0.001, k = 3)"
mean,0.892909,0.906667,0.906667
std,0.008801,0.005774,0.005774


In [55]:
# Accuracy on test set

pd.DataFrame([acc_vgg16, acc_cl_closed_set, acc_tl_closed_set], index = ['VGG16', 'Contrastive_Loss', 'Triplet_Loss'], columns = ['Accuracy']).T

,VGG16,Contrastive_Loss,Triplet_Loss
Accuracy,0.93,0.92,0.94


# Summary on Open Set

In [56]:
# After tuning hps (d_t), the best accuracy on val set.

best_cl = table_cl_open_set.loc['0.5']
best_tl = table_tl_open_set.loc['0.6']

pd.DataFrame({'Contrastive_Loss(d_t = )': best_cl, 
              'Triplet_Loss(d_t = )': best_tl
             })

,Contrastive_Loss(d_t = ),Triplet_Loss(d_t = )
mean,0.723333,0.720000
std,0.011547,0.034641


In [57]:
# Accuracy on test set

pd.DataFrame([acc_cl_open_set, acc_tl_open_set], index = ['Contrastive_Loss', 'Triplet_Loss'], columns = ['Accuracy']).T

,Contrastive_Loss,Triplet_Loss
Accuracy,0.77,0.76
